Planar Robots with joints
=========================

**Topic**: Pathplanning for planar robots with rotary joints

Version | Author | Remark
------------ | ------------- | -------------
0.1 | Gergely Sotí | Base concept implemented and prepared
0.2 | Björn Hein | Minor textual additions and corrections


This worksheet **explains the basic concepts when applying path planning algorithm to planar robots with rotary joints** 


**ATTENTION**: You will have to have installed the python library sympy: https://www.sympy.org/en/index.html

License is based on Creative Commons: Attribution-NonCommercial 4.0 International (CC BY-NC 4.0) (pls. check: http://creativecommons.org/licenses/by-nc/4.0/)


Important links are:

    General Info: http://www.python.org
    Python tutorial http://www.python.org/doc/tut/
    NetworkX http://networkx.github.io/
    NumPy and SciPy Guide http://docs.scipy.org/
    Matplotlib gallery http://matplotlib.sourceforge.net/gallery.html

Remember that:

    you have to press ctrl-return or shift-return, to execute the code in the code sections, only then the variables are "generated" and can be used
    you can execute the whole notebook by Cell->runAll

**Attention**: package sympy is needed(!)

In [ ]:
#!pip install sympy

In [ ]:
import sys
sys.path.append(".")

%load_ext autoreload
%autoreload 2

import math

In [ ]:
from IPPlanarManipulator import PlanarJoint, PlanarRobot

Kinematic chains
====
Joint D-H parameters simplified for planar case:

$\theta$: joint rotation

a: joint displacement (=1.5 in this simple setup)

d and $\alpha$ are omitted.

A Planar robot with 2 joints. (We use 2 joints in this example, as the sampling in the BasicPRM class samples in 2 dimensions)

In [ ]:
j = PlanarJoint()
# Printing joint at 0.0
print(j.get_transform())

# Printing joint at 0.8 (rad)
j.move(0.8)
print(j.get_transform())

#Printing joint at pi 
j.move(math.pi/2)
print(j.get_transform())

# Planar with two joints and length 1.5 of segments.
r = PlanarRobot(n_joints=2)
r.move([0.0, math.pi/2])
print(r.get_transforms())

Kinematic chain collision checker
========
Maps joint positions to task space positions of the joints, and uses those for collision checking. Lines in joint space correspond to a movement in task space. To check those for collisions, the joint space line is discretized and each point is checked for collision in task space.  

In [ ]:
from IPEnvironment import CollisionChecker
import numpy as np
import copy

def interpolate_line(startPos, endPos, step_l):
    steps = []
    line = np.array(endPos) - np.array(startPos)
    line_l = np.linalg.norm(line)
    step = line / line_l * step_l
    n_steps = np.floor(line_l / step_l).astype(np.int32)
    c_step = np.array(startPos)
    for i in range(n_steps):
        steps.append(copy.deepcopy(c_step))
        c_step += step
    if not (c_step == np.array(endPos)).all():
        steps.append(np.array(endPos))
    return steps
    

class KinChainCollisionChecker(CollisionChecker):
    def __init__(self, kin_chain, scene, limits=[[-3.0, 3.0], [-3.0, 3.0]], statistic=None, fk_resolution=0.1):
        super(KinChainCollisionChecker, self).__init__(scene, limits, statistic)
        self.kin_chain = kin_chain
        self.fk_resolution = fk_resolution
        self.dim = self.kin_chain.dim

    def getDim(self):
        return self.dim
        
    
    def pointInCollision(self, pos):
        self.kin_chain.move(pos)
        joint_positions = self.kin_chain.get_transforms()
        self.dim = 2
        for i in range(1, len(joint_positions)):
            if self.segmentInCollision(joint_positions[i-1], joint_positions[i]):
                self.dim = self.kin_chain.dim
                return True
        self.dim = self.kin_chain.dim
        return False
    
    def lineInCollision(self, startPos, endPos):
        assert (len(startPos) == self.getDim())
        assert (len(endPos) == self.getDim())
        steps = interpolate_line(startPos, endPos, self.fk_resolution)
        for pos in steps:
            if self.pointInCollision(pos):
                return True
        return False
    
    def segmentInCollision(self, startPos, endPos):
        assert (len(startPos) == self.getDim())
        assert (len(endPos) == self.getDim())
        for key, value in self.scene.items():
            if value.intersects(LineString([(startPos[0], startPos[1]), (endPos[0], endPos[1])])):
                return True
        return False
    
    def drawObstacles(self, ax, inWorkspace=False):
        if inWorkspace:
            for key, value in self.scene.items():
                plotting.plot_polygon(value, add_points=False, color='red', ax=ax)

In [ ]:
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, LineString
from shapely import plotting
#from descartes.patch import PolygonPatch

Robot visualization in task space
====
Joint positions are mapped to the position of joints in task space. A line ins drawn between the joint postions.

In [ ]:
def planarRobotVisualize(kin_chain, ax):
    joint_positions = kin_chain.get_transforms()
    for i in range(1, len(joint_positions)):
        xs = [joint_positions[i-1][0], joint_positions[i][0]]
        ys = [joint_positions[i-1][1], joint_positions[i][1]]
        ax.plot(xs, ys, color='g')

In [ ]:

obst = dict()
obst["obs1"] = LineString([(-2, 0), (-0.8, 0)]).buffer(0.5)
obst["obs2"] = LineString([(2, 0), (2, 1)]).buffer(0.2)
obst["obs3"] = LineString([(-1, 2), (1, 2)]).buffer(0.1)

## create environment with robot and obstacle
environment = KinChainCollisionChecker(r, obst,fk_resolution=.2)

fig_local = plt.figure(figsize=(14, 7))
## right figure for robot start position
ax1 = fig_local.add_subplot(1, 2, 1)
ax1.set_xlim([-3,3])
ax1.set_ylim([-3,3])
environment.drawObstacles(ax1, True)
start_joint_pos = [2.0, 0.5]
r.move(start_joint_pos)
planarRobotVisualize(r, ax1)
## left figure robot goal position
ax2 = fig_local.add_subplot(1, 2, 2)
ax2.set_xlim([-3,3])
ax2.set_ylim([-3,3])
environment.drawObstacles(ax2, True)
end_joint_pos = [-2.0, -0.5]
r.move(end_joint_pos)
planarRobotVisualize(r, ax2)

plt.show()

In [ ]:
environment.pointInCollision(start_joint_pos)

In [ ]:
environment.pointInCollision(end_joint_pos)

Graph visualization with Animation
======

In [ ]:
import matplotlib.animation
from IPython.display import HTML

matplotlib.rcParams['animation.embed_limit'] = 64
def animateSolution(planner, environment, solution, visualizer):
    _planner = planner
    _environment = environment
    _solution = solution
    _prmVisualizer = visualizer
    
    fig_local = plt.figure(figsize=(14, 7))
    ax1 = fig_local.add_subplot(1, 2, 1)
    ax2 = fig_local.add_subplot(1, 2, 2)
    ## get positions for solution
    solution_pos = [_planner.graph.nodes[node]['pos'] for node in _solution]
    ## interpolate to obtain a smoother movement
    i_solution_pos = [solution_pos[0]]
    for i in range(1, len(solution_pos)):
        segment_s = solution_pos[i-1]
        segment_e = solution_pos[i]
        i_solution_pos = i_solution_pos + interpolate_line(segment_s, segment_e, 0.1)[1:]
    ## animate
    frames = len(i_solution_pos)
    
    def animate(t):
        ## clear taks space figure
        ax1.cla()
        ## fix figure size
        ax1.set_xlim([-3,3])
        ax1.set_ylim([-3,3])
        ## draw obstacles
        _environment.drawObstacles(ax1, inWorkspace = True)
        ## update robot position
        pos = i_solution_pos[t]
        r.move(pos)
        planarRobotVisualize(r, ax1)
    
        ## clear joint space figure
        ax2.cla()
        ## draw graph and path
        _prmVisualizer(_planner, solution, ax2)
        ## draw current position in joint space
        ax2.scatter(i_solution_pos[t][0], i_solution_pos[t][1], color='r', zorder=10, s=250)

    ani = matplotlib.animation.FuncAnimation(fig_local, animate, frames=frames)
    html = HTML(ani.to_jshtml())
    display(html)
    plt.close()


# Path planning

## Basic PRM

In [ ]:
import IPBasicPRM
import IPVISBasicPRM

In [ ]:
## create BasicPRM with the kinematic chain collision checker
basicPRM = IPBasicPRM.BasicPRM(environment)
basicConfig = dict()
basicConfig["radius"] = 5
basicConfig["numNodes"] = 200

start = [start_joint_pos]
goal  = [end_joint_pos]
## plan path from start ot goal
solution = basicPRM.planPath(start, goal, basicConfig)
print(start, goal)
print(solution)

In [ ]:
animateSolution(basicPRM, environment, solution, IPVISBasicPRM.basicPRMVisualize)

## Visibility PRM

In [ ]:
import IPVisibilityPRM
import IPVISVisibilityPRM


In [ ]:
visbilityConfig = dict()
visbilityConfig["ntry"] = 50

visibilityPRM = IPVisibilityPRM.VisPRM(environment)
start = [start_joint_pos]
goal  = [end_joint_pos]
## plan path from start ot goal
solution = visibilityPRM.planPath(start, goal, visbilityConfig)
print(solution)

In [ ]:
animateSolution(visibilityPRM, environment, solution, IPVISVisibilityPRM.visibilityPRMVisualize)

## Lazy PRM

In [ ]:
import IPLazyPRM
import IPVISLazyPRM

In [ ]:
lazyPRM = IPLazyPRM.LazyPRM(environment)
lazyConfig = dict()
lazyConfig["initialRoadmapSize"] = 20
lazyConfig["updateRoadmapSize"]  = 5 
lazyConfig["kNearest"] = 15
lazyConfig["maxIterations"] = 15

start = [start_joint_pos]
goal  = [end_joint_pos]
## plan path from start ot goal
solution = lazyPRM.planPath(start, goal, lazyConfig)
print(solution)


In [ ]:
animateSolution(lazyPRM, environment, solution, IPVISLazyPRM.lazyPRMVisualize)